In [1]:
# adapted from:
# - https://towardsdatascience.com/scikit-learn-for-text-analysis-of-amazon-fine-food-reviews-ea3b232c2c1b
# - https://github.com/shshnk158/Amazon-Fine-Food-Reviews-Prediciton/blob/master/Logistic%20Regression%20Assignment.ipynb

import inline as inline
# %matplotlib inline

import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
import string
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk import SnowballStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


define required funtion(s)

In [2]:
import re
def process(text):
        """ Input: A series of senctences [w1,w2,w3,....]"""

        temp_1 = []
        snow = nltk.stem.SnowballStemmer('english')
        i = 0
        n = len(text)
        for sentence in text:
            sentence = sentence.lower()
            cleanr = re.compile('<.*?>')
            sentence = re.sub(cleanr, ' ', sentence)  # Removing HTML tags
            sentence = re.sub(r'[?|!|\'|"|#]', r'', sentence)
            sentence = re.sub(r'[.|,|)|(|\|/]', r' ', sentence)  # Removing Punctuations

            words = [snow.stem(word) for word in sentence.split() if
                     word not in stopwords.words('english')]  # Splitting the words
            temp_1.append(words)  # and doing stemming
            print("{0:.2f} %".format((i / n) * 100), end='\r')
            i = i + 1

        text = temp_1

        sent = []
        for row in text:
            seq = ''
            for word in row:
                seq = seq + ' ' + word
            sent.append(seq)

        text = sent

        """ Output: Series of sentences after processing """
        return text


In [3]:
# df = pd.read_csv('../data/Reviews.csv')
# df.head()
# df.dropna(inplace=True)
# keep only reviews that are strongly helpful or unhelpful
# mean_helpfulness = df['HelpfulnessNumerator'].sum()/df['HelpfulnessDenominator'].sum()
# df = df[(df['HelpfulnessDenominator'] > 6)
#         & ((df['HelpfulnessNumerator']/df['HelpfulnessDenominator'] < 0.33)
#              | (df['HelpfulnessNumerator']/df['HelpfulnessDenominator'] > mean_helpfulness))]
# adding helpfulness indicating column of a binary attribute where 1 implies helpful, 0 otherwise
# df['Helpful'] = np.where(df['HelpfulnessNumerator']/df['HelpfulnessDenominator'] > mean_helpfulness, 1, 0)
# filtered_data['Helpful'] = np.where(filtered_data['HelpfulnessNumerator']/filtered_data['HelpfulnessDenominator'] > mean_helpfulness, 1, 0)
# df.head()
# filtered_data.head()
# inspect the amount of negative/unhelpful examples
# print(len(df[df['Helpful'] == 0].index))

X_train = pd.read_csv('../data/X_train.csv')
X_train = X_train['Text']
X_test = pd.read_csv('../data/X_test.csv')
X_test = X_test['Text']
y_train = pd.read_csv('../data/y_train.csv')
y_train = y_train['Helpful']
y_test = pd.read_csv('../data/y_test.csv')
y_test = y_test['Helpful']

In [4]:

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Helpful'], random_state = 0)
# # an index that exists in the pruned dataset
# print('X_train first entry: \n\n', X_train.head(1))#hdenom possible data points 6: 5:36527 4:135201 3:25288
# print('\n\nX_train shape: ', X_train.shape)

some preprocessing

In [5]:
# X_train = process(X_train)
# X_test = process(X_test)

using ngrams

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df = 5, ngram_range = (1,3),max_features=2**16).fit(X_train)
vect.get_feature_names()[::2000]

['00',
 'air tight contain',
 'aspartam price amazon',
 'bi',
 'buy natur',
 'chick pea',
 'complaint wish',
 'cuppa black green',
 'discontinu upset',
 'eat soy jerki',
 'extra chore',
 'flavor good',
 'fyi',
 'grate find',
 'histori mani',
 'intens casual drinker',
 'leafi green',
 'long long',
 'meal cat',
 'muir',
 'ok edibl even',
 'packag open',
 'potenti organ toxic',
 'puppi love new',
 'regular white',
 'say give',
 'size pea',
 'state mind easi',
 'syrup general good',
 'think anyon',
 'turkey can recal',
 'vitamin calcium iron',
 'without doubt']

In [7]:
len(vect.get_feature_names())

65536

In [15]:
X_train_vectorized = vect.transform(X_train)
X_train_vectorized

<23885x65536 sparse matrix of type '<class 'numpy.int64'>'
	with 1969652 stored elements in Compressed Sparse Row format>

In [9]:
X_train_vectorized.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

#Logistic regression

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vectorized, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score
predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))
confusion_matrix(y_test,predictions,normalize='true')


AUC:  0.8096714342991335


array([[0.65987934, 0.34012066],
       [0.04053647, 0.95946353]])

In [12]:
feature_names = np.array(vect.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()
print('Smallest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:-11:-1]]))
f1_score(y_test,predictions)

Smallest Coefs: 
['america' 'might well' 'didnt work' 'drown' 'flavor great' 'cancel' '55'
 'disgust' 'ruin' 'your go']

Largest Coefs: 
['beauti' 'excel' 'satisfi' 'pricey' 'realli good' 'delici' 'outstand'
 'tender' 'beat' 'discontinu']



0.9466171003717472

export the train and test set for easy reuse

In [13]:
import csv
# with open('../data/X_train.csv','wb') as result_file:
#     wr = csv.writer(result_file,dialect='excel')
#     wr.writerows(X_train)
#
# with open('../data/X_test.csv','wb') as result_file:
#     wr = csv.writer(result_file,dialect='excel')
#     wr.writerows(X_test)
X_train_dict = {'Text': X_train}
X_test_dict = {'Text': X_test}

X_train_df = pd.DataFrame(X_train_dict,columns=['Text'])
X_test_df = pd.DataFrame(X_test_dict,columns=['Text'])

X_train_df.to_csv(r'../data/X_train.csv', index= False)
X_test_df.to_csv(r'../data/X_test.csv', index= False)

y_train.to_csv(r'../data/y_train.csv', index=False)
y_test.to_csv(r'../data/y_test.csv', index=False)